In [26]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import sys
import requests
import time
import pandas as pd
import warnings
import datetime
from io import BytesIO
warnings.filterwarnings('ignore')

In [13]:
def Market_cap_csv_down(datatime):
    driver=webdriver.Chrome("C:/pydata/chromedriver.exe")

    url = "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101"
    driver.get(url)
    time.sleep(2)
    
    driver.find_element_by_xpath('//*[@id="trdDd"]').click()
    driver.find_element_by_name('trdDd').send_keys(Keys.CONTROL + "a")
    driver.find_element_by_name('trdDd').send_keys(str(datatime))
    driver.find_element_by_xpath('//*[@id="jsSearchButton"]').click()
    time.sleep(2)
    
    driver.find_element_by_xpath('//*[@id="MDCSTAT015_FORM"]/div[2]/div/p[2]/button[2]/img').click()
    driver.find_element_by_xpath('/html/body/div[2]/section[2]/section/section/div/div/form/div[2]/div[2]/div[2]/div/div[2]/a').click()
    time.sleep(3)

In [64]:
def Market_cap_file(datetime):
    gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    gen_parms = {
            'mktId': 'ALL',
            'trdDd': str(datetime),
            'share': '1',
            'csvxls_isNo': 'false',
            'name': 'fileDown',
            'url': 'dbms/MDC/STAT/standard/MDCSTAT01501'
    }
    headers = {
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'
    }
    
    request_otp = requests.get(url=gen_url, params=gen_parms, headers=headers)
    
    down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    data = {
    'code': request_otp.content
    }
    
    down_data = requests.post(url = down_url, data=data, headers=headers)

    Market_capital = pd.read_csv(BytesIO(down_data.content), encoding='cp949')
    Market_capital = Market_capital[['종목코드', '종목명', '시장구분', '종가', '시가총액']]
    Market_capital['날짜'] = datetime
    
    return Market_capital

In [65]:
Market_cap_file('20220120')

,종목코드,종목명,시장구분,종가,시가총액,날짜
0,060310,3S,KOSDAQ,3100,143441690300,20220120
1,095570,AJ네트웍스,KOSPI,5230,244880602850,20220120
2,006840,AK홀딩스,KOSPI,18650,247067012650,20220120
3,054620,APS홀딩스,KOSDAQ,11750,239632096750,20220120
4,265520,AP시스템,KOSDAQ,24150,369046317150,20220120
...,...,...,...,...,...,...
2603,000547,흥국화재2우B,KOSPI,30050,4615680000,20220120
2604,000545,흥국화재우,KOSPI,8160,6266880000,20220120
2605,003280,흥아해운,KOSPI,2305,554179392195,20220120
2606,037440,희림,KOSDAQ,8290,115417317750,20220120


In [77]:
df = pd.DataFrame()
for date in date_period:
    date = str(date).replace('-','')[0:8]
    market_cap = Market_cap_file(date)
    df = pd.concat([df, market_cap], axis=0, ignore_index=True)

In [92]:
def mar_cap_tocsv(date):
    end_date = date
    date_period = pd.date_range(end=end_date, periods=14, freq='B')
    df = pd.DataFrame(columns=['종목코드', '종목명', '시장구분', '종가', '시가총액', '날짜'])
    
    for date in date_period:
        date = str(date).replace('-','')[0:8]
        market_cap = Market_cap_file(date)
        df = pd.concat([df, market_cap], axis=0, ignore_index=True)
    
    return df.to_csv(f'C:/Users/ksw07/Desktop/시가총액/{date}.csv', index=False)
    

In [87]:
date_df = ['20110331', '20110630', '20110930', '20111231',
          '20120331', '20120630', '20120930', '20121231',
          '20130331', '20130630', '20130930', '20131231',
          '20140331', '20140630', '20140930', '20141231',
          '20150331', '20150630', '20150930', '20151231',
          '20160331', '20160630', '20160930', '20161231',
          '20170331', '20170630', '20170930', '20171231',
          '20180331', '20180630', '20180930', '20181231',
          '20190331', '20190630', '20190930', '20191231',
          '20200331', '20200630', '20200930', '20201231',
          '20210331', '20210630', '20210930', '20211231',
          '20220331']

In [93]:
for date in date_df:
    mar_cap_tocsv(date)

In [184]:
def mar_cap_mean(f_name):
    df = pd.read_csv('C:/Users/ksw07/Desktop/시가총액/' + f_name)
    stock_code = df['종목코드'].unique()
    mar_cap_mean_df = pd.DataFrame(columns=['종목코드', '종목명', f_name[:-4]])
    
    for code in stock_code:
        stock_name = df[df['종목코드'] == code]['종목명'].unique()[0]
        mar_cap_mean = df[df['종목코드'] == code]['시가총액'].mean()
        df2 = pd.DataFrame([[code, stock_name, mar_cap_mean]], columns=['종목코드', '종목명', f_name[:-4]])
        mar_cap_mean_df = pd.concat([mar_cap_mean_df, df2], axis=0, ignore_index = True)
        
    return mar_cap_mean_df.to_csv(f'C:/Users/ksw07/Desktop/시가총액/분기별평균/{f_name}', index=False)

In [169]:
import os
os.chdir('C:/Users/ksw07/Desktop/시가총액')
print(os.listdir())

['20110331.csv', '20110630.csv', '20110930.csv', '20111230.csv', '20120330.csv', '20120629.csv', '20120928.csv', '20121231.csv', '20130329.csv', '20130628.csv', '20130930.csv', '20131231.csv', '20140331.csv', '20140630.csv', '20140930.csv', '20141231.csv', '20150331.csv', '20150630.csv', '20150930.csv', '20151231.csv', '20160331.csv', '20160630.csv', '20160930.csv', '20161230.csv', '20170331.csv', '20170630.csv', '20170929.csv', '20171229.csv', '20180330.csv', '20180629.csv', '20180928.csv', '20181231.csv', '20190329.csv', '20190628.csv', '20190930.csv', '20191231.csv', '20200331.csv', '20200630.csv', '20200930.csv', '20201231.csv', '20210331.csv', '20210630.csv', '20210930.csv', '20211231.csv', '20220331.csv', '분기별평균']


In [185]:
for file in os.listdir()[:-1]:
    mar_cap_mean(file)    

In [186]:
os.chdir('C:/Users/ksw07/Desktop/시가총액/분기별평균')

In [187]:
print(os.listdir())

['20110331.csv', '20110630.csv', '20110930.csv', '20111230.csv', '20120330.csv', '20120629.csv', '20120928.csv', '20121231.csv', '20130329.csv', '20130628.csv', '20130930.csv', '20131231.csv', '20140331.csv', '20140630.csv', '20140930.csv', '20141231.csv', '20150331.csv', '20150630.csv', '20150930.csv', '20151231.csv', '20160331.csv', '20160630.csv', '20160930.csv', '20161230.csv', '20170331.csv', '20170630.csv', '20170929.csv', '20171229.csv', '20180330.csv', '20180629.csv', '20180928.csv', '20181231.csv', '20190329.csv', '20190628.csv', '20190930.csv', '20191231.csv', '20200331.csv', '20200630.csv', '20200930.csv', '20201231.csv', '20210331.csv', '20210630.csv', '20210930.csv', '20211231.csv', '20220331.csv']


In [197]:
df1 = pd.read_csv('C:/Users/ksw07/Desktop/시가총액/분기별평균/' + '20110331.csv')

df2 = pd.read_csv('C:/Users/ksw07/Desktop/시가총액/분기별평균/' + '20110630.csv')

pd.merge(df1, df2, on=['종목코드', '종목명'], how='outer')

,종목코드,종목명,20110331,20110630
0,069110,3H,4.421505e+10,4.780265e+10
1,060310,3S,1.500834e+11,1.480114e+11
2,900010,3노드디지탈,9.693474e+10,6.403669e+10
3,038120,AD모터스,4.673474e+10,6.283227e+10
4,013340,AJS,7.526157e+10,3.904602e+10
...,...,...,...,...
2047,110500,쓰리피시스템,NaN,2.785960e+10
2048,069640,엠케이트렌드,NaN,9.365000e+10
2049,019770,탑금속,NaN,9.245440e+10
2050,071840,하이마트,NaN,1.319671e+12


In [201]:
df1 = pd.read_csv('C:/Users/ksw07/Desktop/시가총액/분기별평균/' + '20110331.csv')
for file in os.listdir()[1:]:
    df2 = df2 = pd.read_csv('C:/Users/ksw07/Desktop/시가총액/분기별평균/' + file)
    df1 = pd.merge(df1, df2, on=['종목코드', '종목명'], how='outer')

In [212]:
df1.rename(columns = {'20110331':'2011_1분기', '20110630':'2011_2분기', '20110930':'2011_3분기', '20111230':'2011_4분기',
          '20120330':'2012_1분기', '20120629':'2012_2분기', '20120928':'2012_3분기', '20121231':'2012_4분기',
          '20130329':'2013_1분기', '20130628':'2013_2분기', '20130930':'2013_3분기', '20131231':'2013_4분기',
          '20140331':'2014_1분기', '20140630':'2014_2분기', '20140930':'2014_3분기', '20141231':'2014_4분기',
          '20150331':'2015_1분기', '20150630':'2015_2분기', '20150930':'2015_3분기', '20151231':'2015_4분기',
          '20160331':'2016_1분기', '20160630':'2016_2분기', '20160930':'2016_3분기', '20161230':'2016_4분기',
          '20170331':'2017_1분기', '20170630':'2017_2분기', '20170929':'2017_3분기', '20171229':'2017_4분기',
          '20180330':'2018_1분기', '20180629':'2018_2분기', '20180928':'2018_3분기', '20181231':'2018_4분기',
          '20190329':'2019_1분기', '20190628':'2019_2분기', '20190930':'2019_3분기', '20191231':'2019_4분기',
          '20200331':'2020_1분기', '20200630':'2020_2분기', '20200930':'2020_3분기', '20201231':'2020_4분기',
          '20210331':'2021_1분기', '20210630':'2021_2분기', '20210930':'2021_3분기', '20211231':'2021_4분기',
          '20220331':'2022_1분기'}, inplace=True)

In [214]:
df1.to_csv('C:/Users/ksw07/Desktop/시가총액.csv')

,Unnamed: 0,종목코드,종목명,2011_1분기,2011_2분기,2011_3분기,2011_4분기,2012_1분기,2012_2분기,2012_3분기,...,2019_4분기,2020_1분기,2020_2분기,2020_3분기,2020_4분기,2021_1분기,2021_2분기,2021_3분기,2021_4분기,2022_1분기
0,0,069110,3H,4.421505e+10,4.780265e+10,4.631109e+10,3.723739e+10,3.341377e+10,2.787366e+10,3.579909e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,060310,3S,1.500834e+11,1.480114e+11,3.503794e+11,9.867575e+11,8.105189e+11,6.818519e+11,4.246380e+11,...,1.233659e+11,7.758085e+10,1.125983e+11,9.734207e+10,9.983493e+10,1.094778e+11,1.452264e+11,1.782295e+11,1.518773e+11,1.518201e+11
2,2,900010,3노드디지탈,9.693474e+10,6.403669e+10,6.025486e+10,5.531356e+10,6.666784e+10,4.846599e+10,5.162168e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,038120,AD모터스,4.673474e+10,6.283227e+10,5.068938e+10,3.961572e+10,2.506364e+10,2.417696e+10,1.974917e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,013340,AJS,7.526157e+10,3.904602e+10,4.085304e+10,4.158156e+10,4.818040e+10,3.632788e+10,3.888645e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4273,4273,388720,유일로보틱스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.373462e+11
4274,4274,366030,공구우먼,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.285167e+10
4275,4275,396300,세아메카닉스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.945212e+11
4276,4276,417310,코람코더원리츠,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.209880e+11
